In [4]:
from dataset import OxfordPets
from tensorflow import keras
from unet import get_model
import random
import os

In [5]:
input_dir = "data/images/"
target_dir = "data/annotations/trimaps/"
img_size = (160, 160)
num_classes = 3
batch_size = 32

In [6]:
input_img_paths = sorted(
    [os.path.join(input_dir, fname) for fname in os.listdir(input_dir) if fname.endswith(".jpg")])
target_img_paths = sorted([
    os.path.join(target_dir, fname)
    for fname in os.listdir(target_dir)
    if fname.endswith(".png") and not fname.startswith(".")
])

In [7]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

In [8]:
# Build model
model = get_model(img_size, num_classes)

2022-11-11 01:18:17.833707: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-11 01:18:17.833738: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-11 01:18:17.833754: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bigger-notebook-0): /proc/driver/nvidia/version does not exist
2022-11-11 01:18:17.833962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Split our img paths into a training and a validation set
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

In [10]:
# Instantiate data Sequences for each split
train_gen = OxfordPets(batch_size, img_size, train_input_img_paths, train_target_img_paths)
val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [11]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

In [12]:
callbacks = [keras.callbacks.ModelCheckpoint("oxford_segmentation.h5", save_best_only=True)]

In [13]:
# Train the model, doing validation at the end of each epoch.
epochs = 15
model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

Epoch 1/15
199/199 [==============================] - 632s 3s/step - loss: 0.8801 - val_loss: 2.2404
Epoch 2/15
199/199 [==============================] - 625s 3s/step - loss: 0.5055 - val_loss: 2.0224
Epoch 3/15
199/199 [==============================] - 622s 3s/step - loss: 0.4311 - val_loss: 0.5738
Epoch 4/15
199/199 [==============================] - 622s 3s/step - loss: 0.3821 - val_loss: 0.4151
Epoch 5/15
199/199 [==============================] - 622s 3s/step - loss: 0.3497 - val_loss: 0.4092
Epoch 6/15
199/199 [==============================] - 627s 3s/step - loss: 0.3203 - val_loss: 0.3820
Epoch 7/15
  2/199 [..............................] - ETA: 8:58 - loss: 0.2645 

KeyboardInterrupt: 

In [14]:
model.save('../models/1')

INFO:tensorflow:Assets written to: ../models/1/assets


INFO:tensorflow:Assets written to: ../models/1/assets
